# Introduction to 

-----

In this notebook, we build 

https://github.com/ericmjl/bayesian-analysis-recipes/blob/master/poisson-regression.ipynb

https://gist.github.com/AustinRochford/d83ecc6acd17a632a2b9df788386540b

-----

## Table of Contents

[](#)

[](#)


-----

Before proceeding with the rest of this notebook, we first include the notebook setup code.

-----

In [1]:
# Set up Notebook

% matplotlib inline

# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# These two lines suppress warnings that sometimes
# occur when making visualizations
import warnings
warnings.filterwarnings('ignore')

-----

[[Back to TOC]](#Table-of-Contents)

## XXX YYY

A 

-----
[wcn]: https://en.wikipedia.org/


-----

<font color='red' size = '5'> Student Exercise </font>


In the preceding cells, we introduced the **XYZ**. Now that you have run the notebook, go back and make the following changes to see how the results change.

1. Change the 
2. Try 
3. Try 

-----

## Ancillary Information

The following links are to additional documentation that you might find helpful in learning this material. Reading these web-accessible documents is completely optional.

1. Wikipedia article on [XXX YYY][ZZZ]

-----

https://jakevdp.github.io/blog/2014/06/14/frequentism-and-bayesianism-4-bayesian-in-python/

http://nbviewer.jupyter.org/github/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/blob/master/Chapter1_Introduction/Ch1_Introduction_PyMC3.ipynb

[ffl]: http://blog.fastforwardlabs.com/2017/01/30/the-algorithms-behind-probabilistic-programming.html

[fflrn]: http://fastforwardlabs.github.io/pre/#/probability/1000000-50/2016/all_boroughs/all_neighborhoods?explainer=true&_k=3dirsi

https://livebook.manning.com/#!/book/practical-probabilistic-programming/chapter-1/1

https://en.wikipedia.org/wiki/Probabilistic_programming_language

http://www.pl-enthusiast.net/2014/09/08/probabilistic-programming/

http://nbviewer.jupyter.org/github/markdregan/Bayesian-Modelling-in-Python/blob/master/Section%200.%20Introduction.ipynb

http://nbviewer.jupyter.org/github/markdregan/Bayesian-Modelling-in-Python/blob/master/Section%201.%20Estimating%20model%20parameters.ipynb

http://austinrochford.com/posts/2017-10-02-bayes-param-survival.html

http://nbviewer.jupyter.org/github/jdwittenauer/ipython-notebooks/blob/master/notebooks/misc/ProbablisticProgramming.ipynb

https://gist.github.com/AustinRochford/d83ecc6acd17a632a2b9df788386540b

http://localhost:8888/notebooks/test/probabilistic_ml.ipynb

http://www.americansocceranalysis.com/home/2017/11/6/whats-the-point-of-practicing-penalties




[ZZZ]: https://en.wikipedia.org/

**&copy; 2017: Robert J. Brunner at the University of Illinois.**

This notebook is released under the [Creative Commons license CC BY-NC-SA 4.0][ll]. Any reproduction, adaptation, distribution, dissemination or making available of this notebook for commercial use is not allowed unless authorized in writing by the copyright holder.

[ll]: https://creativecommons.org/licenses/by-nc-sa/4.0/legalcode